In [2]:
AUDIO_DIR = 'data/audio'
AUDIO_FILEPATTERN = AUDIO_DIR + '/*.wav'
SAVE_DIR = 'models/ddsp-solo-instrument'

In [3]:
import glob
import os

if not glob.glob(AUDIO_FILEPATTERN):
    raise ValueError('No audio files found. Please use the previous cell to '
                    'upload.')
                    
TRAIN_TFRECORD = 'data/train.tfrecord'
TRAIN_TFRECORD_FILEPATTERN = TRAIN_TFRECORD + '*'

!ddsp_prepare_tfrecord \
    --input_audio_filepatterns=$AUDIO_FILEPATTERN \
    --output_tfrecord_path=$TRAIN_TFRECORD \
    --num_shards=10 \
    --alsologtostderr

C:\Users\TheoA\Anaconda3\envs\tflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
I1128 14:54:01.772249 30736 environments.py:376] Default Python SDK image for environment is apache/beam_python3.8_sdk:2.43.0
I1128 14:54:02.109045 30736 translations.py:714] ==================== <function annotate_downstream_side_inputs at 0x0000018866998EE0> ====================
I1128 14:54:02.112082 30736 translations.py:714] ==================== <function fix_side_input_pcoll_coders at 0x0000018866999040> ====================
I1128 14:54:02.113583 30736 translations.py:714] ==================== <function pack_combiners at 0x0000018866999550> ====================
I1128 14:54:02.116101 30736 translations.py:714] ==================== <function lift_combiners at 0x00000188669995E0> ====================
I1128 14:54:02.116101 30736 